In [34]:
import pandas as pd

df = pd.DataFrame(columns=['Date', 'Ticker', 'Ticker Average Title Sentiment', 'Ticker Average Body Sentiment', 'Total Average Sentiment', "Next Day Ticker Price Change Direction", "5 Day Ticker Price Change Direction"])
df

,Date,Ticker,Ticker Average Title Sentiment,Ticker Average Body Sentiment,Total Average Sentiment,Next Day Ticker Price Change Direction,5 Day Ticker Price Change Direction


In [35]:
import numpy as np 
import pandas as pd 
import json
from matplotlib import pyplot as plt
with open('saps.json') as f:
  data = json.load(f)

# Displaying Subreddits 
subList = list(data.keys())
# print(subList)

In [36]:
for sub in subList:
    data[sub]['raw']['postData'] = pd.DataFrame(data = np.array(data[sub]['raw']['postData']), columns = ['Ticker','title','text','flair','unix'])
    data[sub]['raw']['postData']['rowInd'] = data[sub]['raw']['postData'].index
    # For each financial data type
    for finType in ['inter','intra']:
    
        # Get a list of post keys
        keyList = list(data[sub]['raw'][finType].keys())
    
        # For each keys, get the financial data for that post
        for key in keyList:
            data[sub]['raw'][finType][key] = np.array(data[sub]['raw'][finType][key])
            data[sub]['raw'][finType][key] = pd.Series(data[sub]['raw'][finType][key][:,1], index = data[sub]['raw'][finType][key][:,0])

In [37]:

# keys = data['Stocks']['md']['inter']['keys']
# sample = data['Stocks']['raw']['inter'][keys[1100]]
# sample_df = pd.DataFrame(data = sample).reset_index()
# sample_df.columns = ['unix','percentChange']
# sample_df['date'] = pd.to_datetime(sample_df['unix'],unit='s', utc=True)
# sample_df['date'] = sample_df['date'].dt.date
# # sample_df = sample_df.set_index('date')
# # sample_df['cumulativeChange'] = sample_df.groupby('date')['percentChange'].cumsum()
# display(sample_df.tail(20))
# # data['Stocks']['raw']['postData'].loc[0]
# display(data['Stocks']['raw']['postData'].loc[1100])
# sample_ticker = data['Stocks']['raw']['postData']['ticker'][1100]
# post_unix = data['Stocks']['raw']['postData']['unix'][1100]
# pd.to_datetime(post_unix,unit='s', utc=True).tz_convert('US/Eastern')

In [38]:
postData_df = pd.DataFrame()
for sub in subList:
    subData_df = data[sub]['raw']['postData']
    subData_df['subreddit'] = sub
    postData_df = pd.concat([postData_df, subData_df], axis = 0)

postData_df['Date'] = pd.to_datetime(postData_df['unix'], unit = 's', utc = True)
postData_df['Date'] = postData_df['Date'].dt.tz_convert('US/Eastern').dt.date
postData_df = postData_df.drop(columns = ['unix'])
postData_df.reset_index(drop = True, inplace = True)

In [39]:
sentiment_df = pd.read_csv('Sentiment.csv', parse_dates=['Date'])
sentiment_df['Date'] = sentiment_df['Date'].apply(lambda x: x.date())
sentiment_df.columns = ['Ticker', 'Date', 'Title Sentiment', 'Body Sentiment']
postData_df = postData_df.merge(sentiment_df, on = ['Ticker', 'Date'], how = 'left')
postData_df['Market Average Sentiment'] = postData_df.groupby(['Date'])[['Title Sentiment', 'Body Sentiment']].transform('mean').mean(axis = 1)
# postData_df['Daily Ticker Title Average Sentiment'] = postData_df.groupby(['Date', 'Ticker'])['Title Sentiment'].transform('mean')
# postData_df['Daily Ticker Body Average Sentiment'] = postData_df.groupby(['Date', 'Ticker'])['Body Sentiment'].transform('mean')

In [40]:
# ! pip install yfinance
import yfinance as yf

model_input = postData_df[['Date', 'Ticker', 'Market Average Sentiment', 'Title Sentiment', 'Body Sentiment']].groupby(['Date', 'Ticker']).agg({'Title Sentiment': 'mean', 'Body Sentiment': 'mean', 'Market Average Sentiment': 'mean'}).reset_index()
model_input.dropna(subset=['Title Sentiment', 'Body Sentiment', 'Market Average Sentiment'], how='all', inplace=True)
start_date = model_input['Date'].min() - pd.Timedelta(days=5)
end_date = model_input['Date'].max() + pd.Timedelta(days=10)

# # Download all data at once
# tickers = model_input['ticker'].unique().tolist()
# ticker_string = " ".join(tickers)  # Convert list of tickers into a space-separated string
# data = yf.download(ticker_string, start=start_date, end=end_date)

In [41]:
# data.to_pickle("stock_data.pkl")

In [42]:
data = pd.read_pickle("stock_data.pkl")

In [43]:
all_dates = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='D'))
all_dates.reset_index(inplace=True)
all_dates.columns = ['Date']
all_dates['Date'] = all_dates['Date'].dt.date

In [44]:
next_day_pct_change = data['Close'].pct_change().__deepcopy__()
next_day_pct_change.dropna(axis = 1, how='all', inplace = True)
next_day_pct_change = next_day_pct_change.shift(-1)
next_day_pct_change.dropna(axis = 0, how='all', inplace = True)
next_day_pct_change.reset_index(inplace = True)
next_day_pct_change['Date'] = pd.to_datetime(next_day_pct_change['Date']).dt.date
next_day_pct_change = next_day_pct_change.merge(all_dates, on = 'Date', how = 'right').fillna(method='bfill')
next_day_pct_change = next_day_pct_change.melt(id_vars = 'Date', var_name = 'Ticker', value_name = 'Next Day Price Change')
next_day_pct_change.dropna(subset=['Next Day Price Change'], inplace=True)
next_day_pct_change['Next Day Price Change Direction'] = (next_day_pct_change['Next Day Price Change'] > 0).astype(int)
# display(next_day_pct_change)

In [45]:
next_5d_pct_change = data['Close'].pct_change(5).__deepcopy__()
next_5d_pct_change.dropna(axis = 1, how='all', inplace = True)
next_5d_pct_change = next_5d_pct_change.shift(-5)
next_5d_pct_change.dropna(axis = 0, how='all', inplace = True)
next_5d_pct_change.reset_index(inplace = True)
next_5d_pct_change['Date'] = pd.to_datetime(next_5d_pct_change['Date']).dt.date
next_5d_pct_change = next_5d_pct_change.merge(all_dates, on = 'Date', how = 'right').fillna(method='bfill')
next_5d_pct_change = next_5d_pct_change.melt(id_vars = 'Date', var_name = 'Ticker', value_name = 'Next 5 Day Price Change')
next_5d_pct_change.dropna(subset=['Next 5 Day Price Change'], inplace=True)
next_5d_pct_change['Next 5 Day Price Change Direction'] = (next_5d_pct_change['Next 5 Day Price Change'] > 0).astype(int)
next_5d_pct_change['Date'] = pd.to_datetime(next_5d_pct_change['Date']).dt.date
# display(next_5d_pct_change)

In [ ]:
next_5d_pct_change = data['Close'].pct_change(10).__deepcopy__()
next_5d_pct_change.dropna(axis = 1, how='all', inplace = True)
next_5d_pct_change = next_5d_pct_change.shift(-10)
next_5d_pct_change.dropna(axis = 0, how='all', inplace = True)
next_5d_pct_change.reset_index(inplace = True)
next_5d_pct_change['Date'] = pd.to_datetime(next_5d_pct_change['Date']).dt.date
next_5d_pct_change = next_5d_pct_change.merge(all_dates, on = 'Date', how = 'right').fillna(method='bfill')
next_5d_pct_change = next_5d_pct_change.melt(id_vars = 'Date', var_name = 'Ticker', value_name = 'Next 5 Day Price Change')
next_5d_pct_change.dropna(subset=['Next 5 Day Price Change'], inplace=True)
next_5d_pct_change['Next 5 Day Price Change Direction'] = (next_5d_pct_change['Next 5 Day Price Change'] > 0).astype(int)
next_5d_pct_change['Date'] = pd.to_datetime(next_5d_pct_change['Date']).dt.date

In [46]:
# model_input['Next Day Ticker Price Change Direction'] = np.nan
# model_input['5 Day Ticker Price Change Direction'] = np.nan
model_input = model_input.merge(next_day_pct_change, how='left', on=['Date', 'Ticker'])
model_input = model_input.merge(next_5d_pct_change, how='left', on=['Date', 'Ticker'])
model_input.drop(columns=['Next Day Price Change', 'Next 5 Day Price Change'], inplace=True)
model_input.dropna(subset=['Next Day Price Change Direction', 'Next 5 Day Price Change Direction'], inplace=True)

# for day in tqdm.tqdm(model_input['Date'].unique(), desc='Processing Days'):
#     day_stock_data_loc = stock_data[stock_data['Date'] >= day].index[0]
#     if stock_data.loc[day_stock_data_loc, 'Date'] != day:
#         day_stock_data_loc -= 1
#     next_day_stock_data = stock_data.loc[day_stock_data_loc + 1]
#     five_days_stock_data = stock_data.loc[day_stock_data_loc + 1:day_stock_data_loc + 5]
#     five_days_stock_data.iloc[:, 1:] = five_days_stock_data.iloc[:, 1:].apply(lambda x: x + 1)
#     five_days_stock_data.iloc[:, 1:] = five_days_stock_data.iloc[:, 1:].cumprod() - 1
#     day_model_input = model_input[model_input['Date'] == day]
#     for ticker in day_model_input['ticker'].unique():
#         model_input.loc[model_input['ticker'] == ticker, 'Next Day Ticker Price Change Direction'] = next_day_stock_data[ticker] > 0
#         model_input.loc[model_input['ticker'] == ticker, '5 Day Ticker Price Change Direction'] = five_days_stock_data[ticker].iloc[-1] > 0

# model_input.dropna(subset=['Next Day Ticker Price Change Direction', '5 Day Ticker Price Change Direction'], inplace=True)
display(model_input)

,Date,Ticker,Title Sentiment,Body Sentiment,Market Average Sentiment,Next Day Price Change Direction,Next 5 Day Price Change Direction
5,2017-03-30,GAIN,0.526700,0.00000,0.263350,1.0,0.0
6,2017-03-30,IDXG,0.526700,0.00000,0.263350,0.0,0.0
7,2017-04-04,NSPR,0.000000,0.00000,0.000000,0.0,0.0
8,2017-05-18,FOSL,-0.648600,0.38180,-0.133400,1.0,0.0
9,2017-07-08,HERO,0.817600,0.00000,0.408800,1.0,1.0
...,...,...,...,...,...,...,...
29064,2020-09-23,NKLA,0.000000,0.59940,0.273513,0.0,0.0
29065,2020-09-23,QYLD,0.000000,0.67575,0.273513,1.0,1.0
29066,2020-09-23,SPY,0.389150,0.18060,0.273513,1.0,1.0
29067,2020-09-23,TSLA,0.000000,0.99770,0.273513,1.0,1.0


In [47]:
# spy_data = yf.download('SPY', start=start_date, end=end_date)
# spy_data = spy_data['Close'].pct_change().reset_index()
# full_date_range = pd.DataFrame({'Date': pd.date_range(start=start_date, end=end_date)})
# full_date_range["Date"] = full_date_range["Date"].dt.date
# spy_data['Date'] = pd.to_datetime(spy_data['Date']).dt.date
# spy_data = full_date_range.merge(spy_data, on='Date', how='left')
# spy_data.fillna(method='ffill', inplace=True)
# spy_data = spy_data.rename(columns={'Close': 'SPY'})
# model_input = model_input.merge(spy_data, on='Date', how='left')
# model_input.dropna(subset=['SPY'], inplace=True)
display(model_input)
model_input.to_csv('model_input.csv', index=False)

,Date,Ticker,Title Sentiment,Body Sentiment,Market Average Sentiment,Next Day Price Change Direction,Next 5 Day Price Change Direction
5,2017-03-30,GAIN,0.526700,0.00000,0.263350,1.0,0.0
6,2017-03-30,IDXG,0.526700,0.00000,0.263350,0.0,0.0
7,2017-04-04,NSPR,0.000000,0.00000,0.000000,0.0,0.0
8,2017-05-18,FOSL,-0.648600,0.38180,-0.133400,1.0,0.0
9,2017-07-08,HERO,0.817600,0.00000,0.408800,1.0,1.0
...,...,...,...,...,...,...,...
29064,2020-09-23,NKLA,0.000000,0.59940,0.273513,0.0,0.0
29065,2020-09-23,QYLD,0.000000,0.67575,0.273513,1.0,1.0
29066,2020-09-23,SPY,0.389150,0.18060,0.273513,1.0,1.0
29067,2020-09-23,TSLA,0.000000,0.99770,0.273513,1.0,1.0


In [48]:
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from cuml.preprocessing import StandardScaler

model_input = pd.read_csv('model_input.csv')
model_input['Date'] = pd.to_datetime(model_input['Date'])
model_input['Date'] = model_input['Date'].dt.date
model_input['Next Day Price Change Direction'] = model_input['Next Day Price Change Direction'].astype(int)
model_input['5 Day Price Change Direction'] = model_input['Next 5 Day Price Change Direction'].astype(int)
model_input.dropna(subset=['Next Day Price Change Direction', '5 Day Price Change Direction'], inplace=True)
model_input.drop(columns=['Date', 'Ticker'], inplace=True)

X = model_input.drop(columns=['Next Day Price Change Direction', '5 Day Price Change Direction'])

scaler = StandardScaler()
X = scaler.fit_transform(X)

y_next_day = model_input['Next Day Price Change Direction']
y_5_day = model_input['5 Day Price Change Direction']
X_train, X_test, y_train, y_test = train_test_split(X, y_next_day, test_size=0.2, random_state=42)
clf = RandomForestClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Next Day Price Change Direction Accuracy: {accuracy}')

X_train, X_test, y_train, y_test = train_test_split(X, y_5_day, test_size=0.2, random_state=42)
clf = RandomForestClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'5 Day Price Change Direction Accuracy: {accuracy}')

/home/peiru/anaconda3/envs/rapids-23.10/lib/python3.9/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Next Day Price Change Direction Accuracy: 0.6460802555084229
5 Day Price Change Direction Accuracy: 1.0


In [50]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Load your data
model_input = pd.read_csv('model_input.csv')
model_input['Date'] = pd.to_datetime(model_input['Date'])
model_input['Date'] = model_input['Date'].dt.date
model_input['Next Day Price Change Direction'] = model_input['Next Day Price Change Direction'].astype(int)
model_input['5 Day Price Change Direction'] = model_input['Next 5 Day Price Change Direction'].astype(int)
model_input.dropna(subset=['Next Day Price Change Direction', '5 Day Price Change Direction'], inplace=True)
model_input.drop(columns=['Date', 'Ticker'], inplace=True)

X = model_input.drop(columns=['Next Day Price Change Direction', '5 Day Price Change Direction']).values
y_next_day = model_input['Next Day Price Change Direction'].values
y_5_day = model_input['5 Day Price Change Direction'].values

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y_next_day = torch.tensor(y_next_day, dtype=torch.long)
y_5_day = torch.tensor(y_5_day, dtype=torch.long)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_next_day, test_size=0.2, random_state=42)

# Create dataloaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 50)  # Assuming input features size is the number of columns in X_train
        self.fc2 = nn.Linear(50, 2)  # Output size is 2 for binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, loss function, and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# Training loop
for epoch in range(50):  # Number of epochs
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan
Epoch 11, Loss: nan
Epoch 12, Loss: nan
Epoch 13, Loss: nan
Epoch 14, Loss: nan
Epoch 15, Loss: nan
Epoch 16, Loss: nan
Epoch 17, Loss: nan
Epoch 18, Loss: nan
Epoch 19, Loss: nan
Epoch 20, Loss: nan
Epoch 21, Loss: nan
Epoch 22, Loss: nan
Epoch 23, Loss: nan
Epoch 24, Loss: nan
Epoch 25, Loss: nan
Epoch 26, Loss: nan
Epoch 27, Loss: nan
Epoch 28, Loss: nan
Epoch 29, Loss: nan
Epoch 30, Loss: nan
Epoch 31, Loss: nan
Epoch 32, Loss: nan
Epoch 33, Loss: nan
Epoch 34, Loss: nan
Epoch 35, Loss: nan
Epoch 36, Loss: nan
Epoch 37, Loss: nan
Epoch 38, Loss: nan
Epoch 39, Loss: nan
Epoch 40, Loss: nan
Epoch 41, Loss: nan
Epoch 42, Loss: nan
Epoch 43, Loss: nan
Epoch 44, Loss: nan
Epoch 45, Loss: nan
Epoch 46, Loss: nan
Epoch 47, Loss: nan
Epoch 48, Loss: nan
Epoch 49, Loss: nan
Epoch 50, Loss: nan
Accuracy: